In [2]:
import pandas as pd
import numpy as np
import os
import utils
from datetime import datetime
import importlib

In [3]:
df = pd.concat([pd.read_csv('hka-aqm-am/' + f.removeprefix('._'), skiprows=1, sep=';', engine='python') for f in os.listdir('hka-aqm-am/')])
df.shape


(608036, 18)

In [5]:
importlib.reload(utils)
performance_df = pd.read_csv('model_performances.csv')
d_model=256
nhead=8
num_layers=5 
dropout_pe=0.15
dropout_encoder=0.15
batch_size=1024
learning_rate=0.00031
epochs=3
aggregation_level = 'quarter_hour'
y_feature = 'CO2'
freq = 15 if aggregation_level == 'quarter_hour' else 30 if aggregation_level == 'half_hour' else 60
window_size = 5 * 60 // freq
for y_feature in ['CO2']:
    
    model, scaler, rmse, mae, me, mape, train_loss, val_loss, n_features = utils.create_multivariate_transformer_model_for_feature(df, d_model=d_model, nhead=nhead, num_layers=num_layers, dropout_pe=dropout_pe, dropout_encoder=dropout_encoder, batch_size=batch_size, learning_rate=learning_rate, epochs=epochs, y_feature=y_feature, aggregation_level=aggregation_level, window_size=window_size)
    performance_df = performance_df.append({'model_name': 'multivariate_transformer','aggregation_level': aggregation_level, 'y_feature': y_feature, 'n_features': n_features, 'rmse': rmse, 'mae': mae, 'me': me, 'mape': mape, 'd_model': d_model, 'nhead': nhead, 'num_layers': num_layers, 'dropout_pe': dropout_pe,'dropout_encoder': dropout_encoder, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs, 'train_loss': train_loss, 'val_loss': val_loss, 'note': '2 decoder Linear layers with LeakyReLu, shuffled training data'}, ignore_index=True)
    performance_df.to_csv('model_performances.csv', index=False)
        

extended data shape from (395686, 27) to (401930, 27)
training data cutoff:  2023-07-14 03:45:00
date_time_rounded    datetime64[ns]
device_id                    object
tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday_sin                 float64
weekday_cos                 float64
month_sin                   float64
month_cos                   float64
time_sin                    float64
time_cos                    float64
semester_SS23               float64
sem

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


          actual   predicted
0     457.000001  432.422542
1     401.000001  410.000218
2     472.000000  471.593813
3     583.000002  596.252556
4     601.000004  622.904292
...          ...         ...
1019  433.000000  426.727638
1020  412.333329  411.173397
1021  450.000000  435.744775
1022  434.000001  438.315401
1023  469.999999  464.471902

[1024 rows x 2 columns]
Epoch 1/25, Training Loss: 0.2294
Epoch 1/25, Validation Loss: 0.1842
          actual   predicted
0     457.000001  440.701184
1     401.000001  409.838766
2     472.000000  476.649004
3     583.000002  577.867889
4     601.000004  595.435962
...          ...         ...
1019  433.000000  436.979284
1020  412.333329  421.580467
1021  450.000000  447.667869
1022  434.000001  445.744292
1023  469.999999  469.981890

[1024 rows x 2 columns]
Epoch 2/25, Training Loss: 0.1392
Epoch 2/25, Validation Loss: 0.1393
          actual   predicted
0     457.000001  432.138962
1     401.000001  388.020915
2     472.000000  452.92665

C:\Users\t-ehm\AppData\Local\Temp\ipykernel_42280\1796540001.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_df = performance_df.append({'model_name': 'multivariate_transformer','aggregation_level': aggregation_level, 'y_feature': y_feature, 'n_features': n_features, 'rmse': rmse, 'mae': mae, 'me': me, 'mape': mape, 'd_model': d_model, 'nhead': nhead, 'num_layers': num_layers, 'dropout_pe': dropout_pe,'dropout_encoder': dropout_encoder, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs, 'train_loss': train_loss, 'val_loss': val_loss, 'note': '2 decoder Linear layers with LeakyReLu, shuffled training data'}, ignore_index=True)


extended data shape from (395686, 27) to (401930, 27)
training data cutoff:  2023-07-14 03:45:00
date_time_rounded    datetime64[ns]
device_id                    object
tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday_sin                 float64
weekday_cos                 float64
month_sin                   float64
month_cos                   float64
time_sin                    float64
time_cos                    float64
semester_SS23               float64
sem

...

extended data shape from (395686, 27) to (401930, 27)
training data cutoff:  2023-07-14 03:45:00
date_time_rounded    datetime64[ns]
device_id                    object
tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday_sin                 float64
weekday_cos                 float64
month_sin                   float64
month_cos                   float64
time_sin                    float64
time_cos                    float64
semester_SS23               float64
sem

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


      actual  predicted
0      36.19  35.675307
1      39.95  39.844160
2      20.85  21.983214
3      35.46  33.372930
4      37.45  36.794921
...      ...        ...
1019   51.86  49.977174
1020   48.35  47.690961
1021   34.06  33.773674
1022   33.59  33.120000
1023   30.13  29.722613

[1024 rows x 2 columns]
Epoch 1/25, Training Loss: 0.0867
Epoch 1/25, Validation Loss: 0.0106
      actual  predicted
0      36.19  36.137265
1      39.95  40.045559
2      20.85  19.598437
3      35.46  35.139621
4      37.45  37.343464
...      ...        ...
1019   51.86  51.470596
1020   48.35  48.907083
1021   34.06  34.783923
1022   33.59  34.019221
1023   30.13  30.354904

[1024 rows x 2 columns]
Epoch 2/25, Training Loss: 0.0267
Epoch 2/25, Validation Loss: 0.0101
      actual  predicted
0      36.19  36.612644
1      39.95  40.702068
2      20.85  20.534586
3      35.46  35.341294
4      37.45  37.511831
...      ...        ...
1019   51.86  51.579367
1020   48.35  49.096182
1021   34.06  34.3

KeyboardInterrupt: 

: 

In [9]:
# Reload the utils module if you have made changes
importlib.reload(utils)

# Read existing model performance records
performance_df = pd.read_csv('model_performances.csv')

# Define LSTM-specific hyperparameters
hidden_dim = 256
num_layers = 5
dropout = 0.3
batch_size = 4096
learning_rate = 0.00031
epochs = 25
aggregation_level = 'quarter_hour'
y_feature = 'CO2'
freq = 15 if aggregation_level == 'quarter_hour' else 30 if aggregation_level == 'half_hour' else 60
window_size = 5 * 60 // freq

# Loop over each feature to create and evaluate the LSTM model
for y_feature in ['VOC', 'hum', 'tmp', 'vis']:
    model, scaler, rmse, mae, me, mape, train_loss, val_loss, n_features = utils.create_multivariate_lstm_model_for_feature(
        df, 
        hidden_dim=hidden_dim, 
        num_layers=num_layers, 
        dropout=dropout, 
        batch_size=batch_size, 
        learning_rate=learning_rate, 
        epochs=epochs, 
        y_feature=y_feature, 
        aggregation_level=aggregation_level, 
        window_size=window_size
    )
    performance_df = performance_df.append({
        'model_name': 'multivariate_lstm',
        'aggregation_level': aggregation_level,
        'y_feature': y_feature,
        'n_features': n_features,
        'rmse': rmse,
        'mae': mae,
        'me': me,
        'mape': mape,
        'hidden_dim': hidden_dim,
        'num_layers': num_layers,
        'dropout': dropout,
        'batch_size': batch_size,
        'learning_rate': learning_rate,
        'epochs': epochs,
        'train_loss': train_loss,
        'val_loss': val_loss,
        'note': 'LSTM model with device_id embedding'
    }, ignore_index=True)
    performance_df.to_csv('model_performances.csv', index=False)

extended data shape from (395686, 27) to (401930, 27)
training data cutoff:  2023-07-14 03:45:00
date_time_rounded    datetime64[ns]
device_id                    object
tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday_sin                 float64
weekday_cos                 float64
month_sin                   float64
month_cos                   float64
time_sin                    float64
time_cos                    float64
semester_SS23               float64
sem

In [ ]:
import torch

device_ids = torch.tensor([0])

device = utils.get_device()

model = utils.load_transformer_model(y_feature="CO2", model_name = 'transformer_multivariate_quarter_hour_26f', device=device)
scaler = utils.load_scaler(y_feature="CO2", model_name = 'transformer_multivariate_quarter_hour_26f')

real_data = utils.load_dataframe(model_name='transformer_multivariate_quarter_hour_26f')
pd.set_option('display.max_columns', None)
real_data.iloc[7:27]

In [ ]:
data_df = real_data.iloc[7:27].drop(columns=['device_id', 'date_time_rounded'])

data_df_scaled = scaler.transform(data_df)

input_data = torch.tensor(data_df_scaled, dtype=torch.float32).view(-1, 20, data_df_scaled.shape[1])
print(input_data.shape)
device_ids = torch.tensor([0])

model.eval()
with torch.no_grad():
    output = model(input_data.to(device), device_ids.to(device))
    print(output)

    prediction = output.cpu().numpy().reshape(-1, 1)
    print(prediction)
    print(prediction.shape)
    zeroes_for_scaler = np.zeros((prediction.shape[0], 25))

    zeroes_for_scaler[:, 2] = prediction  # Insert predicted values into the correct column
    print(zeroes_for_scaler)
    inverse_transformed = scaler.inverse_transform(zeroes_for_scaler)
    predicted_unscaled = inverse_transformed[:, 2].round(2)
    print(predicted_unscaled)
    